In [1050]:
import circuitgraph as cg
import pandas as pd
import networkx as nx
import os
import math
from circuitgraph.io import genus_flops, generic_flop, dc_flops
from circuitgraph import BlackBox

### Making Stdcell library dict

In [1051]:
stdcell45nmlib = pd.read_excel("../Data/stdcell45nm.xlsx")

In [1052]:
stdcell45nmlib

,MACROS,AREA
0,nand,3.2851
1,or,3.2851
2,buf,3.2851
3,not,3.2851
4,and,3.2851
5,nor,4.6930
6,fflopd,7.0395
7,fflond,7.0385
8,xor,4.2237
9,xnor,4.2237


In [1053]:
stdlib45nm_dict = dict(zip(stdcell45nmlib['MACROS'], stdcell45nmlib['AREA']))

In [1054]:
stdlib45nm_dict

{'nand': 3.2851,
 'or': 3.2851,
 'buf': 3.2851,
 'not': 3.2851,
 'and': 3.2851,
 'nor': 4.693,
 'fflopd': 7.0395,
 'fflond': 7.0385,
 'xor': 4.2237,
 'xnor': 4.2237}

## Making Grid Cell Dimensional dict

In [1055]:
circuit_area_df = pd.read_excel('../Data/grid_size.xlsx')
circuit_area_df

,CIRCUIT,grid_width,grid_height,total_gates,n_and,n_or,n_nand,n_nor,n_not,n_buf,n_fflopd
0,c17.v,6,6,6,0,0,6,0,0,0,0
1,c3540.v,84,84,1669,498,92,298,68,490,223,0
2,c5315.v,98,98,2307,718,214,454,27,581,313,0
3,c6288.v,110,110,2416,256,0,0,2128,32,0,0
4,c7552.v,120,120,3513,776,244,1028,54,876,535,0
5,s13207.v,80,80,1367,0,0,502,123,262,281,199
6,s38417.v,244,244,13452,0,0,7411,576,2491,1512,1462
7,s38584.v,229,229,11864,0,0,6436,899,2116,1254,1159
8,i2c.v,92,92,2037,1156,32,0,0,834,15,0
9,div.v,637,637,100985,56536,93,0,0,44356,0,0


In [1056]:
# (circuit_area_df['grid_height'].where(circuit_area_df.CIRCUIT == 'c17.v'))[0]

In [1443]:
circuit_area_dict = dict(zip(circuit_area_df['CIRCUIT'], zip(circuit_area_df['grid_width'], circuit_area_df['grid_height'])))

In [1444]:
circuit_information = {row['CIRCUIT']: row.drop('CIRCUIT').to_dict() for _, row in circuit_area_df.iterrows()}


In [1275]:
def add_to_dictionary(circuit_name):
    aman = ChipDesignGraph(f'../Data/Circuits/{circuit_name}')
    c = aman.get_circuit()
    
    ls_and = set()
    ls_or = set()
    ls_nand = set()
    ls_nor = set()
    ls_not = set()
    ls_buf = set()
    ls_fflopd = set()
    
    
    for node in c.nodes():
        if c.type(node) == 'and':
            ls_and.add(node)
        if c.type(node) == 'or':
            ls_or.add(node)
        if c.type(node) == 'nand':
            ls_nand.add(node)
        if c.type(node) == 'nor':
            ls_nor.add(node)
        if c.type(node) == 'not':
            ls_not.add(node)
        if c.type(node) == 'buf':
            ls_buf.add(node)
        if c.type(node) == 'bb_input':
            if '_reg.D' in node:
                ls_fflopd.add(node)
            
        
    cr = {circuit_name : {
        'nodes' : c.nodes(),
        'n_and' : ls_and,
        'n_or' : ls_or,
        'n_nand' : ls_nand,
        'n_nor' : ls_nor,
        'n_not' : ls_not,
        'n_buf' : ls_buf,
        'n_fflopd' : ls_fflopd,
    }}
    
    circuit_dict.update(cr)
    circuit_dict

In [1305]:
add_to_dictionary('sqrt.v')

In [1495]:
df = pd.DataFrame(circuit_dict,index=['nodes', 'n_and', 'n_or', 'n_nand','n_nor','n_not','n_buf','n_fflopd'])

In [1496]:
df

,c17.v,c3540.v,c5315.v,c6288.v,c7552.v,s13207.v,s38417.v,s38584.v,i2c.v,voter.v,square.v,multiplier.v,div.v,memctrl.v,sqrt.v
nodes,"{N1, N2, N22, N6, N10, N19, N7, N23, N11, N16,...","{N4062, N4115, N1598, N1404, N3100, N3511, N40...","{N7686, N3724, N7940, N3511, N1962, N7658, N60...","{N6230, N3724, N1404, N4001, N3511, N5706, N61...","{N9242, N8246, N9569, N9235, N6230, N3724, N14...","{w58, g1327_reg.D, w19, g7293, n_431, g1098, n...","{g322_reg.Q, g1867, n_1394, n_1053, n_2942, n_...","{n_1394, n_1053, n_2942, n_2659, g6589_reg.CK,...","{po139, n_431, n320, n_1150, n1034, n1372, n_1...","{n_1053, n5537, n_13652, n_6912, n5194, n_6707...","{n_1053, n_16243, n5537, n_13652, n_14278, n_6...","{n_21799, n_1053, n_16243, n5537, n_13652, n_6...","{n_26644, n_40008, n_1053, n_16243, n55295, n_...","{n_26644, n_1053, n_16243, n_14278, n34231, n_...","{n_1053, n_16243, n5537, n_13652, n_14278, n_6..."
n_and,{},"{N2417, N4549, N3664, N3277, N2440, N1586, N20...","{N6096, N7686, N6107, N3724, N3511, N6103, N34...","{N666, N879, N828, N642, N1005, N735, N663, N1...","{N8262, N4641, N3035, N1962, N1813, N4549, N31...",{},{},{},"{n1487, n1236, n1616, po139, n960, n328, n1036...","{n4761, n5537, n7757, n10975, n10248, n3042, n...","{n17438, n477, n4761, n5537, n7757, n320, n109...","{n5537, n5194, n23503, n12126, n22485, n20909,...","{n47660, n55295, n5537, n29083, n34231, n40882...","{n47660, n5537, n29083, n34231, n40882, n5194,...","{n5537, n12126, n22485, n20909, n23760, n24265..."
n_or,{},"{N2697, N1991, N2769, N2659, N3415, N1987, N26...","{N3849, N3861, N6566, N3745, N6925, N7555, N64...",{},"{N9242, N9089, N3891, N8898, N2654, N10381, N1...",{},{},{},"{po138, po123, po015, po119, po079, po075, po1...",{maj},"{\asquared[102], \asquared[66], \asquared[94],...",{},"{\remainder[37], \quotient[2], \remainder[9], ...","{po0840, po0832, po0720, po0461, po0940, po071...","{\asqrt[54], \asqrt[12], \asqrt[47], \asqrt[25..."
n_nand,"{N10, N19, N23, N22, N11, N16}","{N1821, N4078, N4740, N5221, N5309, N4643, N46...","{N6947, N5309, N4264, N7940, N7658, N6024, N68...",{},"{N11236, N9569, N9235, N6540, N6107, N9597, N1...","{w58, n_431, n_607, n_483, w106, n_244, w174, ...","{n_1394, n_1053, n_6912, n_1734, n_5927, n_670...","{n_1394, n_1053, n_2942, n_2659, n_6912, n_649...",{},{},{},{},{},{},{}
n_nor,{},"{N4255, N3394, N4258, N1870, N3399, N3396, N36...","{N4575, N4646, N6553, N6872, N6494, N4605, N46...","{N6230, N3724, N1404, N4001, N3511, N5706, N61...","{N8134, N9149, N3658, N4871, N9146, N1721, N36...","{n_438, n_109, n_117, n_309, n_501, n_500, n_2...","{n_6056, n_4385, n_2858, n_8285, n_7154, n_666...","{n_5168, n_5883, n_8812, n_1894, n_6054, n_346...",{},{},{},"{n320, n3120, n24704, n22105, n23417, n4971, n...",{},"{n_38230, n_37626, n_38289, n_37650, n_38341, ...",{n_16617}
n_not,{},"{N4115, N1598, N1404, N3100, N3035, N5201, N30...","{N6831, N3145, N2903, N5091, N1660, N5138, N74...","{N1374, N1384, N1343, N1400, N1380, N1378, N13...","{N8246, N6096, N6230, N6228, N10544, N11110, N...","{w211, w19, w, n_49, n_305, w93, n_190, n_144,...","{n_8047, n_2942, n_2659, w311, w19, n_6545, n_...","{n_1972, n_2862, w19, n_7154, n_9185, n_5972, ...","{n_851, n_431, n_607, n_483, n_294, n_244, n_1...","{n_1394, n_1053, n_2942, n_2659, n_13652, n_69...","{n_1394, n_1053, n_2942, n_2659, n_16243, n_13...","{n_21799, n_22602, n_1394, n_1053, n_2942, n_1...","{n_26644, n_21799, n_40008, n_1053, n_16243, n...","{n_26644, n_21799, n_1053, n_16243, n_13652, n...","{n_1394, n_1053, n_2942, n_2659, n_16243, n_13..."
n_buf,{},"{N4062, N3965, N1384, N3472, N3142, N3511, N26...","{N6331, N1384, N6370, N1433, N1606, N3035, N19...",{},"{N1374, N8760, N3724, N1404, N1160, N3745, N54...","{g4267, g7293, new_g3635_, g1057, g1098, g1494...","{g1867, g2273, g1098, g780, g1358, g6712, g311...","{g2273, g3111, g1624, g17577, g1714, g2169, g5...","{po007, po006, po004, po002, po010, po013, po0...",{},"{\asquared[0], \asquared[1]}",{},{},"{po0046, po1212, po0130, 

In [1516]:
df.to_excel('../Data/circuit_nodes_info.xlsx')

In [1517]:
new_cols = {'Unnamed: 0' : 'Features'}

In [1518]:
df1 = pd.read_excel('../Data/circuit_nodes_info.xlsx')
df1.rename(columns = new_cols, inplace=True)

In [1519]:
# df1.loc[df1['Features'] == 'nodes', 'c17.v'][0]

In [1520]:
df1

,Features,c17.v,c3540.v,c5315.v,c6288.v,c7552.v,s13207.v,s38417.v,s38584.v,i2c.v,voter.v,square.v,multiplier.v,div.v,memctrl.v,sqrt.v
0,nodes,"{'N1', 'N2', 'N22', 'N6', 'N10', 'N19', 'N7', ...","{'N4062', 'N4115', 'N1598', 'N1404', 'N3100', ...","{'N7686', 'N3724', 'N7940', 'N3511', 'N1962', ...","{'N6230', 'N3724', 'N1404', 'N4001', 'N3511', ...","{'N9242', 'N8246', 'N9569', 'N9235', 'N6230', ...","{'w58', 'g1327_reg.D', 'w19', 'g7293', 'n_431'...","{'g322_reg.Q', 'g1867', 'n_1394', 'n_1053', 'n...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'g658...","{'po139', 'n_431', 'n320', 'n_1150', 'n1034', ...","{'n_1053', 'n5537', 'n_13652', 'n_6912', 'n519...","{'n_1053', 'n_16243', 'n5537', 'n_13652', 'n_1...","{'n_21799', 'n_1053', 'n_16243', 'n5537', 'n_1...","{'n_26644', 'n_40008', 'n_1053', 'n_16243', 'n...","{'n_26644', 'n_1053', 'n_16243', 'n_14278', 'n...","{'n_1053', 'n_16243', 'n5537', 'n_13652', 'n_1..."
1,n_and,set(),"{'N2417', 'N4549', 'N3664', 'N3277', 'N2440', ...","{'N6096', 'N7686', 'N6107', 'N3724', 'N3511', ...","{'N666', 'N879', 'N828', 'N642', 'N1005', 'N73...","{'N8262', 'N4641', 'N3035', 'N1962', 'N1813', ...",set(),set(),set(),"{'n1487', 'n1236', 'n1616', 'po139', 'n960', '...","{'n4761', 'n5537', 'n7757', 'n10975', 'n10248'...","{'n17438', 'n477', 'n4761', 'n5537', 'n7757', ...","{'n5537', 'n5194', 'n23503', 'n12126', 'n22485...","{'n47660', 'n55295', 'n5537', 'n29083', 'n3423...","{'n47660', 'n5537', 'n29083', 'n34231', 'n4088...","{'n5537', 'n12126', 'n22485', 'n20909', 'n2376..."
2,n_or,set(),"{'N2697', 'N1991', 'N2769', 'N2659', 'N3415', ...","{'N3849', 'N3861', 'N6566', 'N3745', 'N6925', ...",set(),"{'N9242', 'N9089', 'N3891', 'N8898', 'N2654', ...",set(),set(),set(),"{'po138', 'po123', 'po015', 'po119', 'po079', ...",{'maj'},"{'\\asquared[102]', '\\asquared[66]', '\\asqua...",set(),"{'\\remainder[37]', '\\quotient[2]', '\\remain...","{'po0840', 'po0832', 'po0720', 'po0461', 'po09...","{'\\asqrt[54]', '\\asqrt[12]', '\\asqrt[47]', ..."
3,n_nand,"{'N10', 'N19', 'N23', 'N22', 'N11', 'N16'}","{'N1821', 'N4078', 'N4740', 'N5221', 'N5309', ...","{'N6947', 'N5309', 'N4264', 'N7940', 'N7658', ...",set(),"{'N11236', 'N9569', 'N9235', 'N6540', 'N6107',...","{'w58', 'n_431', 'n_607', 'n_483', 'w106', 'n_...","{'n_1394', 'n_1053', 'n_6912', 'n_1734', 'n_59...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_69...",set(),set(),set(),set(),set(),set(),set()
4,n_nor,set(),"{'N4255', 'N3394', 'N4258', 'N1870', 'N3399', ...","{'N4575', 'N4646', 'N6553', 'N6872', 'N6494', ...","{'N6230', 'N3724', 'N1404', 'N4001', 'N3511', ...","{'N8134', 'N9149', 'N3658', 'N4871', 'N9146', ...","{'n_438', 'n_109', 'n_117', 'n_309', 'n_501', ...","{'n_6056', 'n_4385', 'n_2858', 'n_8285', 'n_71...","{'n_5168', 'n_5883', 'n_8812', 'n_1894', 'n_60...",set(),set(),set(),"{'n320', 'n3120', 'n24704', 'n22105', 'n23417'...",set(),"{'n_38230', 'n_37626', 'n_38289', 'n_37650', '...",{'n_16617'}
5,n_not,set(),"{'N4115', 'N1598', 'N1404', 'N3100', 'N3035', ...","{'N6831', 'N3145', 'N2903', 'N5091', 'N1660', ...","{'N1374', 'N1384', 'N1343', 'N1400', 'N1380', ...","{'N8246', 'N6096', 'N6230', 'N6228', 'N10544',...","{'w211', 'w19', 'w', 'n_49', 'n_305', 'w93', '...","{'n_8047', 'n_2942', 'n_2659', 'w311', 'w19', ...","{'n_1972', 'n_2862', 'w19', 'n_7154', 'n_9185'...","{'n_851', 'n_431', 'n_607', 'n_483', 'n_294', ...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_13...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_16...","{'n_21799', 'n_22602', 'n_1394', 'n_1053', 'n_...","{'n_26644', 'n_21799', 'n_40008', 'n_1053', 'n...","{'n_26644', 'n_21799', 'n_1053', 'n_16243', 'n...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_16..."
6,n_buf,set(),"{'N4062', 'N3965', 'N1384', 'N3472', 'N3142', ...","{'N6331', 'N1384', 'N6370', 'N1433', 'N1606', ...",set(),"{'N1374', 'N8760', 'N3724', 'N1404', 'N1160', ...","{'g4267', 'g7293', 'new_g3635_', 'g1057', 'g10...","{'g1867', 'g2273', 'g1098', 'g780', 'g1358', '...","{'g2273', 'g3111', 'g1624', 'g17577', 'g1714',...","

In [1521]:
df1.set_index('Features',inplace=True)

In [1527]:
df1.to_dict()

{'c17.v': {'nodes': "{'N1', 'N2', 'N22', 'N6', 'N10', 'N19', 'N7', 'N23', 'N11', 'N16', 'N3'}",
  'n_and': 'set()',
  'n_or': 'set()',
  'n_nand': "{'N10', 'N19', 'N23', 'N22', 'N11', 'N16'}",
  'n_nor': 'set()',
  'n_not': 'set()',
  'n_buf': 'set()',
  'n_fflopd': 'set()'},
 'c3540.v': {'nodes': "{'N4062', 'N4115', 'N1598', 'N1404', 'N3100', 'N3511', 'N4066', 'N5201', 'N238', 'N4075', 'N851', 'N4067', 'N2440', 'N1586', 'N5242', 'N1833', 'N4786', 'N1117', 'N3703', 'N4656', 'N2158', 'N3855', 'N5007', 'N5295', 'N2243', 'N4329', 'N3384', 'N950', 'N4317', 'N3399', 'N3102', 'N5002', 'N303', 'N2135', 'N3307', 'N2342', 'N2032', 'N4980', 'N5279', 'N1679', 'N4597', 'N4964', 'N270', 'N3534', 'N2476', 'N2038', 'N3241', 'N2967', 'N2977', 'N3537', 'N2638', 'N3001', 'N3023', 'N2141', 'N4116', 'N4940', 'N4952', 'N5058', 'N3426', 'N4527', 'N3393', 'N4330', 'N1348', 'N107', 'N1736', 'N3731', 'N317', 'N3236', 'N1256', 'N5352', 'N3670', 'N3048', 'N1861', 'N1350', 'N4984', 'N3014', 'N3803', 'N3333', 'N44

In [1367]:
class ChipDesignGraph:
    def __init__(self,circuit_name):
        
        self.circuit = cg.from_file(circuit_name, blackboxes= genus_flops + dc_flops)
        self.circuit_name = self.circuit.name+'.v'
        self.grid_size = (circuit_information[self.circuit_name]['grid_width'], circuit_information[self.circuit_name]['grid_height'] )
        self.design_graph = nx.grid_2d_graph(*self.grid_size)
        self.node_attributes = {}
        self.edge_attributes = {}
    

    def get_circuit(self):
        return self.circuit       
        

    def add_node_attributes(self,  node_data):
        """
        Add attributes to each node in the graph.
        node_data: Dictionary mapping node IDs to attribute dictionaries.
        """
        self.node_attributes.update(node_data)
        for node, data in node_data.items():
            self.design_graph.nodes[node].update(data)

    
    def add_edge_attributes(self, edge_data):
        """
        Add attributes to edges in the graph.
        edge_data: Dictionary mapping edge tuples (u, v) to attribute dictionaries.
        """
        self.edge_attributes.update(edge_data)
        for edge, data in edge_data.items():
            self.design_graph.edges[edge].update(data)

    
    def DrawGraph(self):
        return nx.draw_spectral(self.design_graph)
        
        
    